<a href="https://colab.research.google.com/github/ekincanozcelik/YoutubeDataAnalysis/blob/master/Emoji_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Emoji Prediction**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
pip install nltk

In [0]:
import pandas as pd
import numpy as np
import random
import nltk
import re
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score

In [0]:
pip install easybert

     |████████████████████████████████| 81kB 5.0MB/s 
     |████████████████████████████████| 71kB 7.1MB/s 
     |████████████████████████████████| 345.2MB 50kB/s 
     |████████████████████████████████| 3.2MB 43.2MB/s 
     |████████████████████████████████| 368kB 56.7MB/s 
  Created wheel for easybert: filename=easybert-1.0.3-cp36-none-any.whl size=7621 sha256=32df14bfd382130641a9c8e623d6ba9c53de2759afea4631c856da7b85bb46b2
  Stored in directory: /root/.cache/pip/wheels/e4/00/9f/587ad878f58cb7255674e9027c546893e8a921b13cec954510
Successfully built easybert
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorflow-hub 0.7.0
    Uninstalling tensorflow-hub-0.7.0:
      Successfully uninstalled tensorflow-hub-0.7.0
  Found existing installat

In [0]:
from easybert import Bert
bert = Bert("https://tfhub.dev/google/bert_multi_cased_L-12_H-768_A-12/1")

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


**Preprocessing**

In [0]:
#Importing datasets#
#Train set
file1 = open("/content/drive/My Drive/Emoji_prediction/us_train.text", "r+", encoding="utf-8")
train_text = file1.readlines()
file2 = open("/content/drive/My Drive/Emoji_prediction/us_train.labels", "r+", encoding="utf-8")
train_labels = file2.readlines()
#Development set
file3 = open("/content/drive/My Drive/Emoji_prediction/us_dev.text", "r+", encoding="utf-8")
dev_text = file3.readlines()
file4 = open("/content/drive/My Drive/Emoji_prediction/us_dev.labels", "r+", encoding="utf-8")
dev_labels = file4.readlines()
#Test set
file5 = open("/content/drive/My Drive/Emoji_prediction/us_test.text", "r+", encoding="utf-8")
test_text = file5.readlines()
file6 = open("/content/drive/My Drive/Emoji_prediction/us_test.labels", "r+", encoding="utf-8")
test_labels = file6.readlines()

train_set = []
dev_set = []
test_set = []
    
for i in range(len(train_text)):
    train_set.append((train_text[i].replace("\n", ""), train_labels[i].replace("\n", "")))
for i in range(len(dev_text)):
    dev_set.append((dev_text[i].replace("\n", ""), dev_labels[i].replace("\n", "")))
for i in range(len(test_text)):
    test_set.append((test_text[i].replace("\n", ""), test_labels[i].replace("\n", "")))

#Shuffle datasets to avoid overfitting
random.shuffle(train_set)
random.shuffle(test_set)
random.shuffle(dev_set)

In [0]:
print(train_set[:5])
print(dev_set[:5])
print(test_set[:5])
print(len(train_text))

[('Young @user killed it tonight . Proud of you always but boy you impressed me tonight . … ', '9'), ('ZEDS DEAD @user #hardsummermusicfestival #hardsummer2016… ', '1'), ("#Valor launch party #BeautyandPinups #PinupsforVets ️ (@ Pearl's Liquor Bar - @user in West Hollywood, CA) ", '0'), ('Pajama night with the fam! ️ you all. #Christmas #nanashouse #pjs @ Jonesboro, Arkansas ', '0'), ('Workout done!! Thank you lavidafit !!! #fitness #workout… ', '9')]
[('happy Friyay from us @ Indianola High School', '19'), ('Spending mothers day with my number 1 person @ Busch Stadium', '0'), ('@user is privileged to offer our biggest congratulations to each and every deserving…', '0'), ("January! @ Mac's Wood Grilled", '0'), ('Beautiful Miami .. @ Smith &amp; Wollensky Miami Steakhouse', '12')]
[("Family for Life @ Macky's Bayside Bar &amp; Grill", '0'), ('Shots from our pre-Turkey day show @user Patrick Gregory @ Urban Lounge', '10'), ('I forgot my iPhone on stage before @user I so deserved to find 

In [0]:
####TEXT PREPROCESSING####
#Initalizing lemmatizer and creating a list of stopwords
lemmatizer = nltk.stem.WordNetLemmatizer()
nltk.download('stopwords')
nltk.download('wordnet')
stopwords = set(nltk.corpus.stopwords.words("english"))

#Cleans reviews, removes stopwords and lemmatize to put the reviews in a form that machine can perform better
def clean_reviews(string):
    lemmatized_string = ""
    #Remove HTML information from the string
    processed_string = re.compile(r"<[^>]+>").sub(" ", string)
    
    #Remove URLS from the string
    processed_string = re.compile(r"https?://[A-Za-z0-9./]+").sub(" ", processed_string)
    
    #Remove digits and punctuations
    processed_string = re.compile(r"[^a-zA-Z ]").sub(" ", processed_string)
    
    #Lowercase all the words
    processed_string = processed_string.lower()
    
    #Does not append stopwords but appends other strings while lemmatizing them
    for word in processed_string.split():
        if word in stopwords:
            continue
        else:
            lemmatized_string += lemmatizer.lemmatize(word) + " "
    return lemmatized_string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
####DATA PREPROCESSING####
#Appending reviews and its corresponding label to different lists for each set
dev_processed_text = []
dev_processed_label = []
train_processed_text = []
train_processed_label = []
test_processed_text = []
test_processed_label = []
for i in train_set:
    train_processed_text.append((clean_reviews(i[0])))
    train_processed_label.append(i[1])  
for i in test_set:
    test_processed_text.append((clean_reviews(i[0])))
    test_processed_label.append(i[1])
for i in dev_set:
    dev_processed_text.append((clean_reviews(i[0])))
    dev_processed_label.append(i[1])

y_train = train_processed_label    
y_test = test_processed_label
y_dev = dev_processed_label

Feature Extraction


In [0]:
tfidf_vectorizer = TfidfVectorizer(ngram_range = (1,2)) #Intialize TF-IDF vectorizer and ngram features
X_train = tfidf_vectorizer.fit_transform(train_processed_text) #Fit and transform our training set reviews using the created TF-IDF vectorize
X_test = tfidf_vectorizer.transform(test_processed_text) #Fit and transform our test set reviews using the created TF-IDF vectorize



Model Implementation


In [0]:
#Logistic Regression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

#Predict on test set using our model
y_pred = base_model.predict(X_test)

#Calculate precision, recall, f1 score and accuracy of the model
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
accuracy = accuracy_score(y_test, y_pred)

#Print the evaluation measures
print ("Precision: "+str(round(precision,5)))
print ("Recall: "+str(round(recall,5)))
print ("F1-Score: "+str(round(f1,5)))
print ("Accuracy: "+str(round(accuracy,5)))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Precision: 0.31532
Recall: 0.20904
F1-Score: 0.21356
Accuracy: 0.32556


In [0]:
#Create a SVM Classifier
#Initalize SVM CLassifier
svm_clf= sklearn.svm.SVC()

#Train the classifier
svm_clf.fit(X_train, y_train)

#Predict on test set
y_pred = svm_clf.predict(X_test)

#Calculate precision, recall, f1 score and accuracy of the model
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
accuracy = accuracy_score(y_test, y_pred) 

#Print the evaluation measures
print ("Precision: "+str(round(precision,5)))
print ("Recall: "+str(round(recall,5)))
print ("F1-Score: "+str(round(f1,5)))
print ("Accuracy: "+str(round(accuracy,5)))

In [0]:
train_sentence_embedding = []
for i in train_text[0:1]:
  x = bert.embed(i)
  train_sentence_embedding.append(x)
print(train_sentence_embedding)

NameError: ignored